# CHAPTER 4: FEATURE ENGINEERING AND SELECTION
## Temporal data
In this document, I will delve into the crucial aspects of feature engineering in temporal data for machine learning applications. Temporal data, which includes time-related information such as timestamps and time series, presents unique challenges and opportunities for feature extraction and manipulation, making it essential to understand effective techniques in this domain.

Throughout this exploration, we will discuss various strategies and methodologies for feature engineering specifically tailored to temporal data using Python. Leveraging libraries such as Pandas, NumPy, and scikit-learn, we will demonstrate techniques including time-based feature extraction, lag features, rolling statistics, and advanced temporal data processing methods.

#### *Jose Ruben Garcia Garcia*
#### *February 2024*
#### *Reference: Practical Machine Learning Python Problems Solver*

## Feature engineering on temporal data

### Data preparation

In [1]:
import datetime
import numpy as np
import pandas as pd
from dateutil.parser import parse
import pytz

In [2]:
time_stamps = ['2015-03-08 10:30:00.360000+00:00', '2017-07-13 15:45:05.755000-07:00',
               '2012-01-20 22:30:00.254000+05:30', '2016-12-25 00:30:00.000000+10:00']
df = pd.DataFrame(time_stamps, columns=['Time'])
df

,Time
0,2015-03-08 10:30:00.360000+00:00
1,2017-07-13 15:45:05.755000-07:00
2,2012-01-20 22:30:00.254000+05:30
3,2016-12-25 00:30:00.000000+10:00


In [3]:
ts_objs = np.array([pd.Timestamp(item) for item in np.array(df.Time)])
df['TS_obj'] = ts_objs
ts_objs

array([Timestamp('2015-03-08 10:30:00.360000+0000', tz='UTC'),
       Timestamp('2017-07-13 15:45:05.755000-0700', tz='UTC-07:00'),
       Timestamp('2012-01-20 22:30:00.254000+0530', tz='UTC+05:30'),
       Timestamp('2016-12-25 00:30:00+1000', tz='UTC+10:00')],
      dtype=object)

### Date based features

In [6]:
df['Year'] = df['TS_obj'].apply(lambda d: d.year)
df['Month'] = df['TS_obj'].apply(lambda d: d.month)
df['Day'] = df['TS_obj'].apply(lambda d: d.day)
df['DayOfWeek'] = df['TS_obj'].apply(lambda d: d.dayofweek)
df['DayName'] = df['TS_obj'].apply(lambda d: d.day_name()) 
df['DayOfYear'] = df['TS_obj'].apply(lambda d: d.dayofyear)
df['WeekOfYear'] = df['TS_obj'].apply(lambda d: d.weekofyear)
df['Quarter'] = df['TS_obj'].apply(lambda d: d.quarter)

df[['Time', 'Year', 'Month', 'Day', 'Quarter', 
    'DayOfWeek', 'DayName', 'DayOfYear', 'WeekOfYear']]

,Time,Year,Month,Day,Quarter,DayOfWeek,DayName,DayOfYear,WeekOfYear
0,2015-03-08 10:30:00.360000+00:00,2015,3,8,1,6,Sunday,67,10
1,2017-07-13 15:45:05.755000-07:00,2017,7,13,3,3,Thursday,194,28
2,2012-01-20 22:30:00.254000+05:30,2012,1,20,1,4,Friday,20,3
3,2016-12-25 00:30:00.000000+10:00,2016,12,25,4,6,Sunday,360,51


### Time based features

In [7]:
df['Hour'] = df['TS_obj'].apply(lambda d: d.hour)
df['Minute'] = df['TS_obj'].apply(lambda d: d.minute)
df['Second'] = df['TS_obj'].apply(lambda d: d.second)
df['MUsecond'] = df['TS_obj'].apply(lambda d: d.microsecond)
df['UTC_offset'] = df['TS_obj'].apply(lambda d: d.utcoffset())

df[['Time', 'Hour', 'Minute', 'Second', 'MUsecond', 'UTC_offset']]

,Time,Hour,Minute,Second,MUsecond,UTC_offset
0,2015-03-08 10:30:00.360000+00:00,10,30,0,360000,0 days 00:00:00
1,2017-07-13 15:45:05.755000-07:00,15,45,5,755000,-1 days +17:00:00
2,2012-01-20 22:30:00.254000+05:30,22,30,0,254000,0 days 05:30:00
3,2016-12-25 00:30:00.000000+10:00,0,30,0,0,0 days 10:00:00


In [8]:
hour_bins = [-1, 5, 11, 16, 21, 23]
bin_names = ['Late Night', 'Morning', 'Afternoon', 'Evening', 'Night']
df['TimeOfDayBin'] = pd.cut(df['Hour'], 
                            bins=hour_bins, labels=bin_names)
df[['Time', 'Hour', 'TimeOfDayBin']]

,Time,Hour,TimeOfDayBin
0,2015-03-08 10:30:00.360000+00:00,10,Morning
1,2017-07-13 15:45:05.755000-07:00,15,Afternoon
2,2012-01-20 22:30:00.254000+05:30,22,Night
3,2016-12-25 00:30:00.000000+10:00,0,Late Night


In [9]:

df['TZ_info'] = df['TS_obj'].apply(lambda d: d.tzinfo)
df['TimeZones'] = df['TS_obj'].apply(lambda d: list({d.astimezone(tz).tzname() 
                                   for tz in map(pytz.timezone, 
                                                 pytz.all_timezones_set)
                                       if d.astimezone(tz).utcoffset() == d.utcoffset()}))

df[['Time', 'UTC_offset', 'TZ_info', 'TimeZones']]

,Time,UTC_offset,TZ_info,TimeZones
0,2015-03-08 10:30:00.360000+00:00,0 days 00:00:00,UTC,"[GMT, UTC, WET, +00]"
1,2017-07-13 15:45:05.755000-07:00,-1 days +17:00:00,UTC-07:00,"[PDT, -07, MST]"
2,2012-01-20 22:30:00.254000+05:30,0 days 05:30:00,UTC+05:30,"[IST, +0530]"
3,2016-12-25 00:30:00.000000+10:00,0 days 10:00:00,UTC+10:00,"[+10, ChST, AEST]"


In [10]:
df['TimeUTC'] = df['TS_obj'].apply(lambda d: d.tz_convert(pytz.utc))
df['Epoch'] = df['TimeUTC'].apply(lambda d: d.timestamp())
df['GregOrdinal'] = df['TimeUTC'].apply(lambda d: d.toordinal())

df[['Time', 'TimeUTC', 'Epoch', 'GregOrdinal']]

,Time,TimeUTC,Epoch,GregOrdinal
0,2015-03-08 10:30:00.360000+00:00,2015-03-08 10:30:00.360000+00:00,1.425811e+09,735665
1,2017-07-13 15:45:05.755000-07:00,2017-07-13 22:45:05.755000+00:00,1.499986e+09,736523
2,2012-01-20 22:30:00.254000+05:30,2012-01-20 17:00:00.254000+00:00,1.327079e+09,734522
3,2016-12-25 00:30:00.000000+10:00,2016-12-24 14:30:00+00:00,1.482590e+09,736322


In [11]:
curr_ts = datetime.datetime.now(pytz.utc)
# compute days elapsed since today
df['DaysElapsedEpoch'] = (curr_ts.timestamp() - df['Epoch']) / (3600*24)
df['DaysElapsedOrdinal'] = (curr_ts.toordinal() - df['GregOrdinal']) 

df[['Time', 'TimeUTC', 'DaysElapsedEpoch', 'DaysElapsedOrdinal']]

,Time,TimeUTC,DaysElapsedEpoch,DaysElapsedOrdinal
0,2015-03-08 10:30:00.360000+00:00,2015-03-08 10:30:00.360000+00:00,3280.326030,3280
1,2017-07-13 15:45:05.755000-07:00,2017-07-13 22:45:05.755000+00:00,2421.815551,2422
2,2012-01-20 22:30:00.254000+05:30,2012-01-20 17:00:00.254000+00:00,4423.055198,4423
3,2016-12-25 00:30:00.000000+10:00,2016-12-24 14:30:00+00:00,2623.159368,2623


feature engineering in temporal data is a critical aspect of building effective machine learning models for time-related tasks. By carefully designing and selecting temporal features, such as year, month, day, day of the week, and others, we can capture meaningful patterns and relationships in time series data. Additionally, techniques like lag features, rolling statistics, and time-based transformations further enhance the predictive power of our models. 